# The Week3 Task3 of Applied DS Capstone Project

In [2]:
!pip install folium
import folium
import requests
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from sklearn.cluster import KMeans

     |████████████████████████████████| 92kB 18.9MB/s eta 0:00:01


In [3]:
df = pd.read_csv("task2.csv")
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [4]:
print(len(df['Borough'].unique()), df['Borough'].unique().tolist())

10 ['Scarborough', 'North York', 'East York', 'East Toronto', 'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto', 'Mississauga', 'Etobicoke']


*Working with only boroughs that contain TORONTO*

In [5]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.shape

(38, 5)

### Creating map of toronto with neighbours superimposed

In [6]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [8]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

<b>Defining foursquare credentials</b>

In [12]:
# The code was removed by Watson Studio for sharing.

In [13]:
LIMIT, radius = 100, 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    toronto_data.loc[0,'Latitude'], 
    toronto_data.loc[0,'Longitude'], 
    radius, 
    LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=CRDS4XWDD4KPADKBVJP1JE5NW3AYATHVMUCCUC4ZFNIWNJZ5&client_secret=ZBKR4ETVK14UV2SMZRJ1OCPSLAZFF2K2RPBEWFEKWHFVFHTE&v=20180605&ll=43.67684518300007,-79.29522499999995&radius=500&limit=100'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d67d9dcbf7dde002c89facf'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 43.68134518750007,
    'lng': -79.28901466400715},
   'sw': {'lat': 43.672345178500066, 'lng': -79.30143533599274}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [34]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
4,Dip 'n Sip,Coffee Shop,43.678897,-79.297745
5,Upper Beaches,Neighborhood,43.680563,-79.292869


In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
toronto_venues = getNearbyVenues(toronto_data['Neighborhood'],toronto_data['Latitude'],toronto_data['Longitude'])
toronto_venues.shape

The Beaches
Riverdale, The Danforth West
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
North Midtown, The Annex, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

(1740, 7)

In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000
1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.015152,0.015152,0.000000,0.000000,0.000000,0.000000
2,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000
3,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.013889,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.013889,0.000000,0.000000
4,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.010309,0.000000,0.010309,0.010309,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.010309,0.000000,0.000000,0.010309,0.010309,0.010309,0.010309,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.010526,0.000000,0.021053,0.000000,0.000000,...,0.000000,0.010526,0.000000,0.000000,0.000000,0.052632,0.000000,0.042105,0.010526,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.000000,0.012048,0.012048,0.000000,0.000000,0.000000,0.012048,0.000000,0.000000,...,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.012048,0.000000,0.012048


In [40]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2                Hotel  0.04
3                  Bar  0.03
4  American Restaurant  0.03


----Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0         Coffee Shop  0.11
1  Italian Restaurant  0.08
2                 Bar  0.05
3                Café  0.05
4          Restaurant  0.05


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1          Restaurant  0.05
2        Cocktail Bar  0.05
3  Seafood Restaurant  0.05
4              Bakery  0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.08
1                    Café  0.07
2  Furniture / Home Store  0.06
3          Sandwich Place  0.04
4              Restaurant  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         ve

## Running k_means clustering to put it in 5 clusters

In [41]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

In [42]:
toronto_grouped = toronto_grouped.join(df.set_index('Neighborhood'), on='Neighborhood')
toronto_grouped

,Cluster Labels,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,PostalCode,Borough,Latitude,Longitude
0,1,"Adelaide, King, Richmond",0.000000,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.030000,...,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,M5H,Downtown Toronto,43.649515,-79.382503
1,1,"Bathurst Quay, CN Tower, Harbourfront West, Is...",0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015152,...,0.015152,0.015152,0.000000,0.000000,0.000000,0.000000,M5V,Downtown Toronto,43.640815,-79.399538
2,1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,...,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000,M5E,Downtown Toronto,43.645160,-79.373675
3,1,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.027778,0.000000,0.013889,0.000000,...,0.000000,0.027778,0.000000,0.013889,0.000000,0.000000,M6K,West Toronto,43.639410,-79.424362
4,1,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,M7Y,East Toronto,43.648690,-79.385440
5,1,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,M4X,Downtown Toronto,43.668160,-79.366602
6,1,Central Bay Street,0.000000,0.000000,0.010309,0.000000,0.010309,0.010309,0.000000,0.000000,...,0.000000,0.010309,0.010309,0.010309,0.010309,0.000000,M5G,Downtown Toronto,43.656091,-79.384930
7,1,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.010526,0.000000,0.021053,0.000000,...,0.000000,0.052632,0.000000,0.042105,0.010526,0.000000,M5T,Downtown Toronto,43.653570,-79.397249
8,1,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,M6G,Downtown Toronto,43.668781,-79.420710
9,1,Church and Wellesley,0.000000,0.012048,0.012048,0.000000,0.000000,0.000000,0.012048,0.000000,...,0.000000,0.000000,0.012048,0.012048,0.000000,0.012048,M4Y,Downtown Toronto,43.666585,-79.381302


Results show most of neighbourhood belong to the same cluster
Finally, find out what are the top 10 most common venues for each cluster

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_grouped['Latitude'], toronto_grouped['Longitude'], toronto_grouped['Neighborhood'], toronto_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
toronto_grouped_mean = toronto_grouped.groupby('Cluster Labels').mean()
toronto_grouped_mean

,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Latitude,Longitude
Cluster Labels,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,43.698231,-79.392717
1,0.002923,0.000377,0.011327,0.000613,0.004387,0.000322,0.002694,0.006073,0.004963,0.003421,...,0.001837,0.00599,0.001098,0.009352,0.00159,0.005023,0.003038,0.000955,43.662056,-79.389674
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,43.728160,-79.387085
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,43.694785,-79.414405
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,43.623470,-79.391507


## and the final output

In [47]:
toronto_grouped_mean = toronto_grouped_mean.drop(columns = ['Latitude','Longitude'])

x=pd.DataFrame(toronto_grouped_mean).T
rslt = pd.DataFrame(np.zeros((0,5)), columns=['top1','top2','top3','top4','top5'])
for i in x.columns:
    df1row = pd.DataFrame(x.nlargest(5, i).index.tolist(), index=['top1','top2','top3','top4','top5']).T
    rslt = pd.concat([rslt, df1row], axis=0)

rslt.index=toronto_grouped_mean.index
rslt

,top1,top2,top3,top4,top5
Cluster Labels,,,,,
0,Park,Playground,Bank,Building,Garden
1,Coffee Shop,Café,Restaurant,Park,Gym
2,Bus Line,Swim School,Yoga Studio,Afghan Restaurant,American Restaurant
3,Park,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop
4,Harbor / Marina,Café,Music Venue,Yoga Studio,Afghan Restaurant
